In [1]:
import ir_datasets
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
import re
from dateutil.parser import parse
import enchant
import pycountry
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Use English stemmer.
stemmer = SnowballStemmer("english")

 

punctuation_regex=r'[!\"\＄%&\'\(\)\*\+,-\./:;<=>\?\[\\\]\^`{\|}~،“؟”]'
#remove punctuation
def remove_punctuations(text): 

    text = re.sub(punctuation_regex, ' ', str(text));
    return text


#delet_rep
def del_rep_letters(text):
    
    pattern = r"(.)\1{2,}"
    match = re.search(pattern,str(text))
    # re.sub(match.group(),"i",text)
    if(match):
        if(match.start()):
            return re.sub(re.escape(match.group()),text[match.start()]+text[match.start()+1],text)
    else:
        return text





# perform POS tagging
def lemmatize_text(words):
    #words = word_tokenize(words)
    pos_tags = nltk.pos_tag(words)
    

# create a lemmatizer object
    lemmatizer = WordNetLemmatizer()

# lemmatize the words based on their POS tags
    lemmatized_words = []
    for word, tag in pos_tags:
        if tag.startswith('NN'):
        # noun
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='n'))
        elif tag.startswith('VB'):
        # verb
             lemmatized_words.append(lemmatizer.lemmatize(word, pos='v'))
        elif tag.startswith('JJ'):
        # adjective
             lemmatized_words.append(lemmatizer.lemmatize(word, pos='a'))
        elif tag.startswith('R'):
        # adverb
             lemmatized_words.append(lemmatizer.lemmatize(word, pos='r'))
        else:
        # other parts of speech
             lemmatized_words.append(word)
    return lemmatized_words


    
def is_date(string, fuzzy=False):
    try:
        parse(string,fuzzy=fuzzy)
        return True
    except ValueError:
        return False

def Convert(string):
    if is_date(string):
        string= parse(string).strftime("%d-%m-%Y")
    return string    
    
def Date_normlization(text):
    return [Convert(w) for w in text]        
from dateutil import parser
 
date_str1 = '2021-06-01'
date_str2 = '2021/06/01'
date_str3 = 'June 1, 2021'
date_str4 = '01-06-2021'
inputs =['AS', 'CA', 'US',"DDH",date_str1,date_str4,date_str2,date_str3]
print(Date_normlization(inputs))



def is_country(term):
    for country in pycountry.countries:
        if country.alpha_2 == term.upper() or country.alpha_3 == term.upper():
            term=country.name
    return term
input_countries =['AS', 'CA', 'US',"DDH"]
def CountryNormlization(text):
    return [is_country(w) for w in text]
print(CountryNormlization(input_countries))

def stem_words(txt):
    stems = [stemmer.stem(word) for word in txt]
    return stems


def convert_lower_case(data):
    return np.char.lower(data)

def is_proper_noun(tag):
    return tag == 'NNP' or tag == 'NNPS'


import enchant
def remove_non_english_words(text):
    d = enchant.Dict("en_US")
    words = text.split()
    english_words = []
    
    tagged_words = nltk.pos_tag(words)
    
    for word, tag in tagged_words:
        if d.check(word) or is_proper_noun(tag):
            english_words.append(word)
        english_text = ' '.join(english_words)
    return english_text;

['AS', 'CA', 'US', 'DDH', '01-06-2021', '06-01-2021', '01-06-2021', '01-06-2021']
['American Samoa', 'Canada', 'United States', 'DDH']


In [2]:
import re
stemmer = PorterStemmer()
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller



# Apply the lemmatize_text function to your text data
 
      
def Processing (query):
    spell = Speller(lang='en')
    query = spell(query)
    #print("spelled",query)
    query=del_rep_letters(query)
   # query=remove_non_english_words(query)
    
    query=remove_punctuations(query)
    
    tokens = word_tokenize(query)
    #print ("token",tokens)
    
# convert to lower case
    tokens=CountryNormlization(tokens)
    tokens=Date_normlization(tokens)
    print("token2",tokens)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    #print ("lower case",tokens)
   

   
 # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in tokens if not w in stop_words]
    
    words=lemmatize_text(words)
    
  #stemming
    words = stem_words(words)
 #   print (stem_word)  
    return words;
#print(stem_word)


quer="I love running in the park with my dogs! It's so much fun."

print ("processed query",Processing(quer))

token2 ['I', 'love', 'running', 'India', 'the', 'park', 'with', 'Malaysia', 'dogs', 'Italy', 's', 'Somalia', 'much', 'fun']
processed query ['love', 'run', 'india', 'park', 'malaysia', 'dog', 'itali', 'somalia', 'much', 'fun']


In [13]:
dataset = ir_datasets.load("beir/quora/dev")
df=pd.DataFrame(dataset.docs)
#print df befor process
df

,doc_id,text
0,1,What is the step by step guide to invest in sh...
1,2,What is the step by step guide to invest in sh...
2,3,What is the story of Kohinoor (Koh-i-Noor) Dia...
3,4,What would happen if the Indian government sto...
4,5,How can I increase the speed of my internet co...
...,...,...
522926,537929,What's this coin?
522927,537930,What is the approx annual cost of living while...
522928,537931,I am having little hairfall problem but I want...
522929,537932,What is like to have sex with cousin?


In [6]:
def process(df):
    #remove stop_ 
    stop_words = stopwords.words('english') 
    df['text_copy'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    #delet rep
    df['text_copy']=df['text_copy'].apply(lambda x:  del_rep_letters(x))
    #remove punctuation
    df['text_copy']=df['text_copy'].apply(lambda x: remove_punctuations(x))
    df['text_copy']=df['text_copy'].apply(lambda x: remove_non_english_words(x))

    #toknize
    df['text_copy']=df['text_copy'].apply(lambda x: word_tokenize(x))
    df['text_copy']=df['text_copy'].apply(lambda x: CountryNormlization(x))
    df['text_copy']=df['text_copy'].apply(lambda x: Date_normlization(x))
    df['text_copy']=df['text_copy'].apply(lambda x: convert_lower_case(x))

    #lemtize
    df['text_copy']=df['text_copy'].apply(lambda x: lemmatize_text(x))
    #stemize
    df['text_copy'] = df['text_copy'].apply(lambda x: [stemmer.stem(y) for y in x])
    #df.to_csv("data.csv")

In [14]:
process(df)
df.to_csv("c:/tmp/per.csv")

In [15]:
#after process
df

,doc_id,text,text_copy
0,1,What is the step by step guide to invest in sh...,"[what, step, step, guid, invest, share, market..."
1,2,What is the step by step guide to invest in sh...,"[what, step, step, guid, invest, share, market]"
2,3,What is the story of Kohinoor (Koh-i-Noor) Dia...,"[what, stori, kohinoor, koh, i, noor, diamond]"
3,4,What would happen if the Indian government sto...,"[what, would, happen, indian, govern, steal, k..."
4,5,How can I increase the speed of my internet co...,"[how, i, increas, speed, internet, connect, us..."
...,...,...,...
522926,537929,What's this coin?,"[what, s, coin]"
522927,537930,What is the approx annual cost of living while...,"[what, approx, annual, cost, live, studi, uic,..."
522928,537931,I am having little hairfall problem but I want...,"[i, littl, hairfal, problem, i, want, use, hai..."
522929,537932,What is like to have sex with cousin?,"[what, like, sex, cousin]"


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
import numpy as np

def dummy(tokens):
    return tokens


vectorizer = TfidfVectorizer(tokenizer=dummy, preprocessor=dummy)
X = vectorizer.fit_transform(df['text_copy'].values)
#sparse_matrix = csr_matrix(X)
df1 = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
#df1.to_csv("inverted_index.csv")

In [20]:
#inverted index _
from collections import defaultdict

inv_indx = defaultdict(list)
for term in df1.columns:
    indices= df1[df1[term] > 0].index.tolist()
    inv_indx[term]=indices

In [21]:
#print in_index for 10000 docs
for term , indices in inv_indx.items():   
    print(f"{term}: {indices}")

#: [217, 1500, 1501, 3544, 8395, 8652, 9022]
$: [391, 485, 1845, 1846, 1956, 2601, 2647, 2963, 3441, 3442, 3546, 3838, 3839, 4664, 4665, 5158, 5159, 5320, 6241, 6880, 6881, 9368, 9547, 9548]
0: [473, 1470, 1471, 1506, 1733, 1830, 1870, 2647, 3706, 3707, 5313, 5700, 6793, 6814, 6815, 7407, 8001, 8264, 9463, 9508]
00: [83, 84, 1538, 1845, 1846, 2397, 2601, 2602, 3130, 3441, 3442, 4338, 4548, 5947, 6445, 6658, 6659, 7572, 9123, 9547, 9548, 9751, 9752]
04: [6438]
07904378172: [6156]
1: [196, 197, 217, 293, 296, 472, 473, 701, 718, 841, 842, 930, 983, 1118, 1470, 1471, 1483, 1585, 1639, 1640, 1680, 1830, 2016, 2017, 2073, 2323, 2324, 2359, 2360, 2444, 2461, 2462, 2863, 3047, 3089, 3450, 3539, 3706, 3707, 3985, 4095, 4244, 4245, 4460, 4799, 4984, 4985, 5022, 5068, 5159, 5189, 5295, 5300, 5301, 5313, 5319, 5856, 5947, 6241, 6415, 6588, 6814, 6815, 6893, 6979, 7228, 7229, 7329, 7380, 7381, 7502, 7548, 7549, 7561, 7608, 7609, 7630, 7924, 8138, 8265, 8621, 8782, 8984, 8987, 9002, 9370, 9427, 942

like: [26, 59, 63, 64, 78, 110, 130, 159, 190, 230, 254, 312, 347, 348, 382, 383, 403, 404, 422, 427, 529, 545, 613, 628, 629, 633, 634, 667, 714, 715, 785, 870, 871, 881, 966, 970, 972, 1054, 1058, 1095, 1096, 1102, 1151, 1152, 1205, 1360, 1492, 1529, 1530, 1619, 1620, 1637, 1705, 1706, 1713, 1714, 1717, 1825, 1841, 1842, 1872, 1942, 1979, 2030, 2035, 2050, 2053, 2056, 2123, 2143, 2161, 2195, 2196, 2210, 2211, 2231, 2232, 2267, 2282, 2285, 2286, 2293, 2294, 2349, 2350, 2385, 2386, 2398, 2448, 2470, 2496, 2497, 2501, 2529, 2547, 2548, 2556, 2607, 2608, 2637, 2660, 2702, 2731, 2732, 2745, 2746, 2767, 2857, 2891, 2921, 2946, 2974, 3222, 3288, 3417, 3426, 3452, 3487, 3488, 3559, 3560, 3618, 3626, 3654, 3710, 3730, 3731, 3784, 3821, 4099, 4103, 4106, 4108, 4176, 4181, 4182, 4259, 4290, 4291, 4317, 4321, 4341, 4369, 4403, 4500, 4534, 4600, 4620, 4699, 4724, 4725, 4736, 4768, 4769, 4836, 4837, 4848, 4852, 4887, 4896, 5000, 5001, 5018, 5024, 5067, 5075, 5166, 5171, 5183, 5200, 5207, 5243, 524

mayer: [6254]
mba: [1310, 2534, 3204, 3205, 3495, 3864, 3890, 4712, 5465, 6146, 7984, 8150, 8182, 8359, 8360, 9313]
mbb: [6427, 7037, 7038, 7638, 7639, 7665, 7666, 7764, 8579]
mbp: [5022]
mc: [4693]
mca: [5631]
mccain: [5211]
mcdonald: [566, 5588]
mcdowel: [1282, 1283]
mcgill: [2775]
mcu: [4564]
mdi: [5047, 5048]
mdm: [9203, 9204]
mdr: [3934, 3935]
me: [10, 11, 73, 74, 620, 631, 736, 737, 785, 821, 1014, 1015, 1036, 1037, 1401, 1402, 1566, 1690, 1715, 1716, 1758, 1795, 1981, 2033, 2143, 2223, 2348, 2543, 2563, 2701, 2759, 2760, 2794, 2866, 2867, 3154, 3205, 3210, 3226, 3294, 3537, 3958, 4131, 4402, 4403, 4837, 4854, 4920, 4958, 5178, 5200, 5825, 5913, 6101, 6102, 6112, 6480, 7170, 7171, 7195, 7635, 7772, 7924, 8024, 8107, 8502, 8731, 8732, 8801, 8861, 8991, 8992, 9163, 9236, 9746, 9871]
meal: [2371, 2922, 2923, 3461, 5906, 6582]
mean: [30, 32, 33, 47, 93, 226, 250, 251, 583, 584, 613, 614, 744, 745, 849, 894, 895, 966, 967, 1108, 1137, 1316, 1342, 1399, 1400, 1579, 1580, 1647, 1966, 22

ز: [7313]
‎: [4312]
‎₹100: [8169]
‎₹500: [8169]
‘: [239, 2278, 9855]
’: [30, 48, 239, 818, 1147, 1179, 2033, 2278, 2790, 2816, 3131, 3141, 3241, 3439, 4030, 4158, 4406, 4407, 5001, 5175, 5993, 6012, 6284, 6637, 6726, 6769, 7241, 7427, 7600, 7623, 7799, 7871, 8258, 8562, 8583, 8598, 8719, 9080, 9318, 9356, 9382, 9389, 9589, 9592, 9855, 9908]
…: [6001, 9824]
‪company‬: [4940]
‪delhi‬: [4940]
₹100: [390, 1747, 2798, 7093, 8246, 8247, 8615, 9668]
₹500: [390, 486, 1747, 2798, 7093, 8246, 8247, 8615, 9668]
−: [7315]
√2: [9137]
∪: [7315]
し: [16]
シ: [16]
一百回: [9409]
분위기: [5482]
﻿: [9440]


In [33]:
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
def cos_similarity(search_query_weights, tfidf_weights_matrix):
    cosine_distance = cosine_similarity(search_query_weights, tfidf_weights_matrix)
    similarity_list = cosine_distance[0]
    return similarity_list

def most_similar(similarity_list):
    most_similar = []
    most_similar2 = []
    for i in range(0, len(similarity_list)):
        if similarity_list[i] > 0.0:
            tmp_index = np.argmax(similarity_list)
            most_similar.append(tmp_index)
            similarity_list[tmp_index] = 0
        
    return most_similar


def match_Query1(que1,Docs):
    print("helooooooo")
    q=Processing(que1)
    search11 = vectorizer.transform([q])
    res3=cos_similarity(search11,X)
    print(res3,len(res3))
    res4=most_similar(res3)    
    return  res4
 

In [34]:
que1="Why do some men spit into the urinal before urinating?"
res4=match_Query1(que1,150)

helooooooo
token2 ['Why', 'Dominican Republic', 'some', 'men', 'spit', 'into', 'the', 'final', 'before', 'printing']
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.] 30


In [37]:
df3=pd.DataFrame(dataset.qrels)
df3

,query_id,doc_id,relevance,iteration
0,318,317,1,0
1,378,377,1,0
2,379,29976,1,0
3,379,380,1,0
4,379,45646,1,0
...,...,...,...,...
7621,537329,537330,1,0
7622,537374,537375,1,0
7623,537736,537737,1,0
7624,537788,537787,1,0


In [38]:
df4 = df3.groupby('query_id')['doc_id'].apply(list).reset_index(name='new')
df4.head(10)

,query_id,new
0,100055,[100054]
1,100192,[100193]
2,10024,"[118, 6953, 33695, 3875, 115735, 12018, 21153,..."
3,100344,"[84729, 255414, 100343, 144671]"
4,100392,[100391]
5,100734,[100735]
6,100742,[91274]
7,100802,[100801]
8,100847,[100846]
9,100883,"[50515, 294582]"


In [39]:
q=pd.DataFrame(dataset.queries)
q.head(10)

,query_id,text
0,318,How does Quora look to a moderator?
1,378,How do I refuse to chose between different thi...
2,379,Did Ben Affleck shine more than Christian Bale...
3,399,What are the effects of demonitization of 500 ...
4,420,Why creativity is important?
5,540,How can I get free gems in Clash of Clans?
6,548,How do you feel when someone upvotes your answ...
7,609,What are the best thriller movie in Hollywood?
8,744,What should someone do to overcome anxiety?
9,784,Why should one hate Shahrukh Khan?


In [40]:
process(q)
q

,query_id,text,text_copy
0,318,How does Quora look to a moderator?,"[how, quora, look, moder]"
1,378,How do I refuse to chose between different thi...,"[how, i, refus, chose, differ, thing, life]"
2,379,Did Ben Affleck shine more than Christian Bale...,"[did, ben, affleck, shine, christian, bale, ba..."
3,399,What are the effects of demonitization of 500 ...,"[what, effect, demonit, 500, 100, rupe, note, ..."
4,420,Why creativity is important?,"[whi, creativ, import]"
...,...,...,...
4995,537329,How do you make instrumentals?,"[how, make, instrument]"
4996,537374,How will be MEO Class 4 exam?,"[how, meo, class, 4, exam]"
4997,537736,How I can remove my hesitation?,"[how, i, remov, hesit]"
4998,537788,Why are transition metals used as catalysts?,"[whi, transit, metal, use, catalyst]"


In [35]:
#function evaluation
def calc_precision(Q_D,R_D):
    #q_d=retrived
    relevant_retrieved_docs = [doc for doc in Q_D if doc in R_D]
    precision = float(len(relevant_retrieved_docs) / len(Q_D))
    return precision

def calc_precision_at_k(Q_D,R_D,k):
   # print(Q_D)
    relevant_docs = set(Q_D).intersection(set(R_D))
    precision =float(len(relevant_docs) / k)
    return precision
#print("\n\n\n")

def calculate_average_precision(retrieved_docs, relevant_docs):
    precision_at_k = []
    num_relevant_docs = len(relevant_docs)
    for i, doc in enumerate(retrieved_docs):
#         print(doc)
        if doc in relevant_docs:
            precision_at_k.append(calc_precision_at_k(retrieved_docs, relevant_docs, i+1))
    if not precision_at_k:
        return 0
    return sum(precision_at_k) / num_relevant_docs

def calculate_map(query_results):
    average_precision = []
    for query, (retrieved_docs, relevant_docs) in query_results.items():
        ap = calculate_average_precision(retrieved_docs, relevant_docs)
        average_precision.append(ap)
    return sum(average_precision) / len(query_results)

def calculate_mrr(query_results):
    reciprocal_ranks = []
    for query, (retrieved_docs, relevant_docs) in query_results.items():
        for i, doc in enumerate(retrieved_docs):
            if doc in relevant_docs:
                reciprocal_ranks.append(1 / (i + 1))
                break
    mrr_score = sum(reciprocal_ranks) / len(query_results)
    return mrr_score

In [42]:
#evaluation
import numpy as np
file1 = open("c:/tmp/mfile.txt", "a+")  # append mode
precision_list = []
recall_list = []
accuracy_list = []
average_precision = []
reciprocal_ranks = []
for i in range(1,len(q)):
    try:
        Xq= vectorizer.transform(q['text_copy'][i])
        res3=cos_similarity(Xq,X)
        res4=most_similar(res3)
        retrived = []
        for t in range(0, len(res4)):  
            retrived.append(df['doc_id'][t])
        relevant=df4['new'][(df4['query_id']==q['query_id'][i])].values[0]
        #calculate precision@10
        ca=calc_precision_at_k(retrived,relevant,10)
        print("\tprecision@10", ca)
         #calculate map
        ap = calculate_average_precision(retrived,relevant)
        average_precision.append(ap)
        true_Positive =len(set(retrived)&set(relevant))
        false_Positive = len(np.setdiff1d(retrived, relevant))
        false=10- true_Positive
        false_Negative = len(np.setdiff1d(relevant, retrived))
        print(i,"\tquery_id:",q['query_id'][i],"\trelevant:",len(relevant)
          ,"\ttrue_Positive:",true_Positive,"\tfalse",false)
        try:
            #calculate mrr
            reciprocal_ranks.append(1 / true_Positive)
            #calculate precission
            precission = (true_Positive) / (true_Positive + false_Positive)
            #calculate recall
            recall = (true_Positive) / (true_Positive + false_Negative)
            pr=(true_Positive) / (true_Positive + false)
            print("\tprecision@10", ca,"\trecall",recall)
            L = ["precission\t",str(precission),"\trecall\t",str(recall),"\n"]
            file1.writelines(L)

        except ZeroDivisionError:
            pass
        
        precision_list.append(precission)
        recall_list.append(recall)
        if false>0 :
           accuracy_list.append(pr)
    except ValueError:
         pass  
file1.close() 

	precision@10 0.1
1 	query_id: 378 	relevant: 1 	true_Positive: 1 	false 9
	precision@10 0.1 	recall 1.0
	precision@10 0.4
2 	query_id: 379 	relevant: 5 	true_Positive: 4 	false 6
	precision@10 0.4 	recall 0.8
	precision@10 0.7
3 	query_id: 399 	relevant: 28 	true_Positive: 7 	false 3
	precision@10 0.7 	recall 0.25
	precision@10 0.1
4 	query_id: 420 	relevant: 1 	true_Positive: 1 	false 9
	precision@10 0.1 	recall 1.0
	precision@10 0.1
5 	query_id: 540 	relevant: 2 	true_Positive: 1 	false 9
	precision@10 0.1 	recall 0.5
	precision@10 0.1
6 	query_id: 548 	relevant: 1 	true_Positive: 1 	false 9
	precision@10 0.1 	recall 1.0
	precision@10 0.1
7 	query_id: 609 	relevant: 2 	true_Positive: 1 	false 9
	precision@10 0.1 	recall 0.5
	precision@10 0.1
8 	query_id: 744 	relevant: 10 	true_Positive: 1 	false 9
	precision@10 0.1 	recall 0.1
	precision@10 0.2
9 	query_id: 784 	relevant: 2 	true_Positive: 2 	false 8
	precision@10 0.2 	recall 1.0
	precision@10 0.1
10 	query_id: 858 	relevant: 2 	tr

KeyboardInterrupt: 

In [43]:
#crawling
def process_url(url):
    if url:
        response = requests.get(url)
        soup = BeautifulSoup(response.content)
    # Find all links on the page
        links = soup.find_all('h')
    # Find all paragraphs on the page
        paragraphs = soup.find_all('p')
    # Create a DataFrame to store the data
        data = pd.DataFrame(columns=['text'])

    # Iterate over the links and paragraphs and store the data in the DataFrame
        for link in links:
            data = data.append({'text': link.text}, ignore_index=True)
        for paragraph in paragraphs:
            data = data.append({ 'text': paragraph.text}, ignore_index=True)
        return data

In [47]:
from urlextract import URLExtract
import numpy as np
import requests
from bs4 import BeautifulSoup
extractor = URLExtract()
da=pd.DataFrame()
import re
URLPATTERN = r'(https?://\S+)'

da['url'] = df['text'][:10000].apply(lambda x: re.findall(URLPATTERN, x))
#da['url'] =  da['url'].apply(lambda x: "".join(x))
list_url = sum(da['url'],[])

In [50]:
#process 2 url
for t in range(0,list_url):
    data=process_url(list_url[t])
    data.to_csv('c:/tmp/cra.csv', sep = '|', mode='a', index= False,header=False)

In [ ]:
#api
import pandas as pd
import json
from flask import Flask, request

# Load the DataFrame from a CSV file
df = pd.read_csv('data.csv')

# Create a Flask app
app = Flask(__name__)

# Define a route to retrieve a book by ID

dataset = ir_datasets.load("antique/train/split200-train")

q=pd.DataFrame(dataset.queries)
df = pd.read_csv('data.csv')
df1,X,vectorizer=Tf_idfData(df)
invertedIndex1=inverted_index(df1)



@app.route('/documents', methods=['POST'])
def upload_query():
    string_data = request.form['string']
    print("string_data",string_data)
    docs=match_Query1(string_data,X,vectorizer,2)
    
    retrieved = pd.DataFrame()
    for doc_id in docs:
        retrieved = retrieved.append(df.loc[df['doc_id'] == doc_id])
    documents = retrieved.to_dict(orient='records')
    
    data = {
    'array':documents}
    print("data",data)
    return json.dumps(data)
    
    
    
    
    
    
    
    
if __name__ == '__main__':
    app.run(host='0.0.0.0')